In [38]:
import os
import re
import json
import unicodedata
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [87]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS",1:"EOS"}
        self.word_count = 2
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.word_count
            self.word2count[word] = 1
            self.index2word[self.word_count] = word
            self.word_count += 1
        else:
            self.word2count[word] += 1
            

In [88]:
def readLangs(lang1,lang2,reverse=False):
    def unicode2ascill(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn'
        )
    def normalizeString(s):
        s = unicode2ascill(s.lower().strip())
        s = re.sub(r'([.!?])',r' \1',s) # \1代表第一个括号内匹配到的值
        s = re.sub(r'[^a-zA-Z.?!]+',r" ",s) # 当在一组方括号里使用^,它表示"非"或"排除"的意思，常常用来剔除某个字符
        return s
    with open('./dataset/en-fr/{}-{}.txt'.format(lang1,lang2),"r",encoding='utf8') as f:
        sentences = [[normalizeString(s) for s in line.strip().split('\t')] for line in f.readlines()]
    if reverse:
        sentences = [list(reversed(sentence)) for sentence in sentences]
        input_lang = Lang(lang2)
        out_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        out_lang = Lang(lang2)
    return input_lang,out_lang,sentences

In [112]:
MAX_LENGTH = 10
eng_prefixe = (
    "i am ","i m ",
    "he is ","he s ",
    "she is ","she s ",
    "they are ", "they re",
    "we are ", "we re ",
    "you are ", "you re " 
    
)
def filterPair(pair):
    return len(pair[0].split(' ')) < MAX_LENGTH and len(pair[1].split(' ')) < MAX_LENGTH and pair[0].startswith(eng_prefixe)
def filterPairs(pairs):
    return [ pair for pair in pairs if filterPair(pair)]

In [113]:
def prepareData(lang1,lang2,reverse=False):
    # 读入pairs
    input_lang,out_lang,pairs = readLangs(lang1,lang2)
    # 筛选pairs
    pairs = filterPairs(pairs)
    # 装载至Lang
    for pair in pairs:
        input_lang.addSentence(pair[0])
        out_lang.addSentence(pair[1])
    print(input_lang.name,input_lang.word_count)
    print(out_lang.name,out_lang.word_count)
    return input_lang,out_lang,pairs

In [114]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

eng 2823
fra 4366
['i m winning .', 'je l emporte .']


#### Encoder
![Encoder架构图](https://pytorch.org/tutorials/_images/encoder-network.png)

In [98]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)
    def forward(self,input_,hidden):
        embedded = self.embedding(input_).view(1,1,-1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output,hidden
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size,device=device)

#### Decoder
![Decoder架构图](https://pytorch.org/tutorials/_images/decoder-network.png)  
使用Encoder最后的output，作为context vector，将该vector作为Decoder的initial hidden state。  
每一次循环，给定一个input token以及hidden state  
在每一个output后，接一个Softmax层，转换为预测下一token的概率

In [108]:
class DecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self,input,hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Teacher-Forcing Mode
Teacher-Forcing Mode，  
参考[Teacher Forcing训练机制](https://blog.csdn.net/qq_30219017/article/details/89090690)

In [121]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5
def trainNoAttention(input_tensor, target_tensor, encoder,
          decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    # 梯度置0
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [116]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [117]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) 
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = trainNoAttention(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [118]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [122]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.word_count, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.word_count).to(device)

trainIters(encoder1, decoder1, n_iters=75000, print_every=5000)

KeyboardInterrupt: 

#### Attention Mechnism

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        

In [ ]:
evaluateRandomly(encoder1, decoder1)

In [ ]:
output_words = evaluate(encoder1, decoder1, "je suis trop froid .")